In [15]:
import os
import boto3
from botocore.exceptions import ClientError
import json
import time
import sagemaker
from sagemaker.pytorch import PyTorchModel
# from sagemaker.pytorch.model import PyTorchModel
from sagemaker import Model

In [ ]:
aws_access_key_id = '' # put your access key id
aws_secret_access_key = '' # put your secret access key
region_name = 'ap-northeast-2'
role = 'arn:aws:iam::565393031158:role/sagemaker-oft-role'
image_uri = '565393031158.dkr.ecr.ap-northeast-2.amazonaws.com/outfittransformer:latest'
path_to_model = 's3://githubsalt-models/model.tar.gz'

In [17]:
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

sagemaker_session = sagemaker.Session(boto_session=session)

In [18]:
model = PyTorchModel(
    image_uri = image_uri,
    model_data = path_to_model,
    role=role
)

In [ ]:
# 배포 코드와 함께 사용
try:
    print("Starting endpoint deployment...")
    predictor = model.deploy(
        instance_type='ml.r5.large',
        initial_instance_count=1,
        endpoint_name='oft-11'
    )
    
except Exception as e:
    print(f"Deployment failed: {str(e)}")

Starting endpoint deployment...
----------

In [ ]:
test_data = {
    "userId" : 'userid',
    "timestamp" : 'timestamp',
    "requiredClothes" : [
        {
            "id" : "top1",
            "name" : "아이보리 케이블 후드 집업 니트",
            "url" : "https://image.msscdn.net/thumbnails/images/goods_img/20231019/3640947/3640947_17170490498222_big.jpg?w=1200",
            "type" : "상의"
        }
        ],
    "clothesList": [
        {
            "id" : "top2",
            "name" : "아이보리 케이블 후드 집업 니트",
            "url" : "https://image.msscdn.net/thumbnails/images/goods_img/20231019/3640947/3640947_17170490498222_big.jpg?w=1200",
            "type" : "상의"
        },
        {
            "id" : "bottom1",
            "name" : "빈티지 캣워싱 와이드 데님 청바지",
            "url" : "https://image.msscdn.net/thumbnails/images/prd_img/20240725/4274045/detail_4274045_17248283394478_big.jpg?w=1200",
            "type" : "하의"
        },
        {
            "id" : "hat1",
            "name" : "챙 넓은 하얀 모자",
            "url" : "https://cafe24.poxo.com/ec01/superbasic/zhfKK2bYYBfSFNO5tt4/vC38gmRBmpf+6sw1vmKhJpvejpW2WNU7nm20zDdMp7Xe0KRvLoGumNgjIq//Cj1HYg==/_/web/product/big/202305/b847779686041a73dc7645f63c8800e1.jpg",
            "type" : "모자"
        }
    ],
    "excludes": [],
    "style": "캐주얼"
}


In [ ]:

def test_endpoint(endpoint_name, test_data):
    try:
        runtime_client = boto3.client('sagemaker-runtime')
        print('예측 시작')
        
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(test_data)
        )
        
        result = json.loads(response['Body'].read().decode())
        print("성공적으로 예측이 완료되었습니다.")
        print("예측 결과:", result)
        return result
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ValidationError':
            print("엔드포인트 이름이 잘못되었습니다.")
        elif e.response['Error']['Code'] == 'ModelError':
            print("모델 실행 중 오류가 발생했습니다.")
        else:
            print("예상치 못한 오류가 발생했습니다:", e)
        return None
    except Exception as e:
        print("오류 발생:", str(e))
        return None


result = test_endpoint('oft-11', test_data)

예측 시작
모델 실행 중 오류가 발생했습니다.
